In [ ]:
import pandas as pd


country = 'England'
competition = 'PremierLeague'
seasons = '2017-2023'


stats = pd.read_csv(f'./{country}/{competition}/{seasons}-stats.csv')
ratings = pd.read_csv(f'./{country}/{competition}/{seasons}-ratings.csv')


print("Files imported successfully")

#### Explore the data to ensure it contains the values we need

In [29]:
print(stats.info())
print(ratings.info())

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2280 entries, 0 to 2279
Data columns (total 14 columns):
 #   Column                Non-Null Count  Dtype 
---  ------                --------------  ----- 
 0   id                    2280 non-null   int64 
 1   season_id             2280 non-null   int64 
 2   date                  2280 non-null   int64 
 3   round                 2280 non-null   int64 
 4   home_team_id          2280 non-null   int64 
 5   home_team             2280 non-null   object
 6   away_team_id          2280 non-null   int64 
 7   away_team             2280 non-null   object
 8   home_score            2280 non-null   int64 
 9   away_score            2280 non-null   int64 
 10  home_half_time_score  2280 non-null   int64 
 11  away_half_time_score  2280 non-null   int64 
 12  home_league_position  2280 non-null   int64 
 13  away_league_position  2280 non-null   int64 
dtypes: int64(12), object(2)
memory usage: 249.5+ KB
None
<class 'pandas.core.frame.DataFrame

#### Loop over team ratings and apply them to each fixture row for home and away teams

In [30]:
for index, row in stats.iterrows():
    rows = team_ratings.loc[(ratings['season_id'] == row['season_id']) & (ratings['fixture_id'] == row['id'])]
    
    if rows.shape[0] != 2:
        raise Exception ('Cannot find 2 rows for a fixture')
    
    for i, rating_row in rows.iterrows():
        if rating_row['team_id'] == row['home_team_id']:
            stats.at[index, 'home_attack_total'] = rating_row['attack_total']
            stats.at[index, 'home_attack_points'] = rating_row['attack_points']
            stats.at[index, 'home_defence_total'] = rating_row['defence_total']
            stats.at[index, 'home_defence_points'] = rating_row['defence_points']
            
        if rating_row['team_id'] == row['away_team_id']:
            stats.at[index, 'away_attack_total'] = rating_row['attack_total']
            stats.at[index, 'away_attack_points'] = rating_row['attack_points']
            stats.at[index, 'away_defence_total'] = rating_row['defence_total']
            stats.at[index, 'away_defence_points'] = rating_row['defence_points']
            

print(stats.info())

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2280 entries, 0 to 2279
Data columns (total 22 columns):
 #   Column                Non-Null Count  Dtype  
---  ------                --------------  -----  
 0   id                    2280 non-null   int64  
 1   season_id             2280 non-null   int64  
 2   date                  2280 non-null   int64  
 3   round                 2280 non-null   int64  
 4   home_team_id          2280 non-null   int64  
 5   home_team             2280 non-null   object 
 6   away_team_id          2280 non-null   int64  
 7   away_team             2280 non-null   object 
 8   home_score            2280 non-null   int64  
 9   away_score            2280 non-null   int64  
 10  home_half_time_score  2280 non-null   int64  
 11  away_half_time_score  2280 non-null   int64  
 12  home_league_position  2280 non-null   int64  
 13  away_league_position  2280 non-null   int64  
 14  home_attack_total     2280 non-null   float64
 15  home_attack_points   

#### Create a new dataframe containing one row per team per fixture

In [32]:
def create_multi_line_stats(df: pd.DataFrame) -> pd.DataFrame:
    df = df.sort_values(['date', 'round'])
    
    base_columns = [
        'id',
        'season_id',
        'round',
        'date',
    ]
    
    home_columns = [
        'home_team',
        'home_score',
        'home_league_position',
        'home_attack_total',
        'home_defence_total',
        'home_attack_points',
        'home_defence_points',
    ]

    away_columns = [
        'away_team',
        'away_score',
        'away_league_position',
        'away_attack_total',
        'away_defence_total',
        'away_attack_points',
        'away_defence_points',
    ]

    column_mappings = [
        'team',
        'goals',
        'league_position',
        'attack_total',
        'defence_total',
        'attack_points',
        'defence_points',
    ]

    home_mapping = {old_column: new_column for old_column, new_column in zip(home_columns, column_mappings)}
    away_mapping = {old_column: new_column for old_column, new_column in zip(away_columns, column_mappings)}

    home_df = df[base_columns + home_columns].rename(columns=home_mapping).assign(at_home=1)
    away_df = df[base_columns + away_columns].rename(columns=away_mapping).assign(at_home=0)

    multi_line = pd.concat([home_df, away_df]).sort_values(by=["date", "round"]).reset_index(drop=True)

    return multi_line


season_results = create_multi_line_stats(results)


print(season_results.info())

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 4560 entries, 0 to 4559
Data columns (total 12 columns):
 #   Column           Non-Null Count  Dtype  
---  ------           --------------  -----  
 0   id               4560 non-null   int64  
 1   season_id        4560 non-null   int64  
 2   round            4560 non-null   int64  
 3   date             4560 non-null   int64  
 4   team             4560 non-null   object 
 5   goals            4560 non-null   int64  
 6   league_position  4560 non-null   int64  
 7   attack_total     4560 non-null   float64
 8   defence_total    4560 non-null   float64
 9   attack_points    4560 non-null   float64
 10  defence_points   4560 non-null   float64
 11  at_home          4560 non-null   int64  
dtypes: float64(4), int64(7), object(1)
memory usage: 427.6+ KB
None


#### For each team in each fixture determine their ranking based on their attack and home ratings. Using this ranking will determine if two teams are closely matched. If they are not closely matched then the presence of skill will be tested.

In [35]:
df = pd.DataFrame()


for ssn in season_results['season_id'].unique():
    season_rows = season_results[season_results['season_id'] == ssn]

    season_rows = season_rows.copy()
    
    for rnd in season_rows['round'].unique():
        round_df = season_rows[season_rows['round'] == rnd]
        
        season_rows.loc[season_rows['round'] == rnd, 'attack_rank'] = round_df['attack_total'].rank(ascending=False, method='min')
        season_rows.loc[season_rows['round'] == rnd, 'defence_rank'] = round_df['defence_total'].rank(ascending=False, method='min')
    
    season_rows['attack_rank'] = season_rows.groupby('team')['attack_rank'].shift(1, fill_value=0)
    season_rows['defence_rank'] = season_rows.groupby('team')['defence_rank'].shift(1, fill_value=0)
        
    df = pd.concat([df, season_rows])


print(df.info())

<class 'pandas.core.frame.DataFrame'>
Index: 4560 entries, 0 to 4559
Data columns (total 14 columns):
 #   Column           Non-Null Count  Dtype  
---  ------           --------------  -----  
 0   id               4560 non-null   int64  
 1   season_id        4560 non-null   int64  
 2   round            4560 non-null   int64  
 3   date             4560 non-null   int64  
 4   team             4560 non-null   object 
 5   goals            4560 non-null   int64  
 6   league_position  4560 non-null   int64  
 7   attack_total     4560 non-null   float64
 8   defence_total    4560 non-null   float64
 9   attack_points    4560 non-null   float64
 10  defence_points   4560 non-null   float64
 11  at_home          4560 non-null   int64  
 12  attack_rank      4560 non-null   float64
 13  defence_rank     4560 non-null   float64
dtypes: float64(6), int64(7), object(1)
memory usage: 534.4+ KB
None


#### Combine single team rows back into fixture rows for data analysis

In [37]:
def create_fixture_rows(df):
    home_columns = {
        "team": "home_team",
        'goals': "home_goals",
        'attack_total': 'home_attack_total',
        'defence_total': 'home_defence_total',
        'attack_rank': 'home_attack_rank',
        'defence_rank': 'home_defence_rank',
        'league_position': 'home_league_position',
    }
    
    away_columns = {
        'team': 'away_team',
        'goals': "away_goals",
        'attack_total': 'away_attack_total',
        'defence_total': 'away_defence_total',
        'attack_rank': 'away_attack_rank',
        'defence_rank': 'away_defence_rank',
        'league_position': 'away_league_position',
    }
    
    home = df[df["at_home"] == 1].rename(columns=home_columns)
    away = df[df["at_home"] == 0].rename(columns=away_columns)
    
    merged = home.merge(away, on=[
        "id", 
        "season_id", 
        "round", 
        "date"
    ], how='left').sort_values(by=["date", "round"])
    
    merged = merged.dropna().drop(columns=[
        'at_home_x', 
        'at_home_y',
        'attack_points_x',
        'defence_points_x',
        'attack_points_y',
        'defence_points_y',
    ])

    merged = merged.sort_values(["date", "round"])
    
    return merged


fixture_rows_df = create_fixture_rows(df)


fixture_rows_df.to_csv(f'~/{country}/{competition}/{seasons}-ratings-processed.csv', index=False)


print(fixture_rows_df.info())

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2280 entries, 0 to 2279
Data columns (total 18 columns):
 #   Column                Non-Null Count  Dtype  
---  ------                --------------  -----  
 0   id                    2280 non-null   int64  
 1   season_id             2280 non-null   int64  
 2   round                 2280 non-null   int64  
 3   date                  2280 non-null   int64  
 4   home_team             2280 non-null   object 
 5   home_goals            2280 non-null   int64  
 6   home_league_position  2280 non-null   int64  
 7   home_attack_total     2280 non-null   float64
 8   home_defence_total    2280 non-null   float64
 9   home_attack_rank      2280 non-null   float64
 10  home_defence_rank     2280 non-null   float64
 11  away_team             2280 non-null   object 
 12  away_goals            2280 non-null   int64  
 13  away_league_position  2280 non-null   int64  
 14  away_attack_total     2280 non-null   float64
 15  away_defence_total   